# GNN on EEG 1

graph:
- nodes: EEG electrodes
- edges: spatial distances between nearest electrodes
- attributes: extracted features

In [2]:
import sys
sys.path.append('../')

In [5]:
from pathlib import Path
import pandas as pd
import networkx as nx 
from sklearn import neighbors


elec_coord_file = Path('./electrode_coordinates.csv')
elec_coord = pd.read_csv(elec_coord_file, index_col=0)

In [6]:
# graph 1 : 4 neerest neighbors
A = neighbors.kneighbors_graph(elec_coord,4).toarray()
A += A.T
A = (A>0).astype(int)
G1 = nx.from_numpy_array(A)
nx.relabel_nodes(G1, elec_coord.reset_index().Electrode.to_dict(), copy=False);

# graph 2 : distance threshold
distances =  

In [48]:
labels = []
Xn = []
Yn = []
Zn = []
for n in G.nodes:
    labels += [n]
    node = elec_coord.loc[n]
    Xn += [node['x']]
    Yn += [node['y']]
    Zn += [node['z']]


Xe = []
Ye = []
Ze = []
for e in G.edges:
    n1 = elec_coord.loc[e[0]]
    n2 = elec_coord.loc[e[1]]
    Xe += [n1['x'], n2['x'], None]
    Ye += [n1['y'], n2['y'], None]
    Ze += [n1['z'], n2['z'], None]

In [55]:
import plotly.graph_objs as go

trace1 = go.Scatter3d(
    x=Xe, y=Ye, z=Ze,
    mode='lines',
    line=dict(color='rgb(0,0,0)', width=1),
    hoverinfo='none',
)

trace2 = go.Scatter3d(
    x=Xn, y=Yn, z=Zn,
    mode = 'markers',
    name = 'electrodes',
    marker = dict(
        symbol='circle',
        size=2,
        line=dict(color='rgb(50,50,50)',width=0.5),
    ),
    text=labels,
    hoverinfo='text',
)

axis=dict(
    showbackground=False,
    showline=False,
    showgrid=False,
    showticklabels=False,
    title='',
)

layout = go.Layout(
    scene = dict(
        xaxis=dict(axis),
        yaxis=dict(axis),
        zaxis=dict(axis),
    )
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.show()